In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import itertools
from IPython.display import display, Markdown

try:
    config = json.load(open("config.json"))
except FileNotFoundError:
    config = {
        "title": "Meziroční analyza: 2017 → 2018",
        "prev": "2017",
        "cur": "2018",
        "prev_file": "UserAttendance-2017.json.anon",
        "cur_file": "UserAttendance-2018.json.anon",
    }
uaprev = json.load(open(config["prev_file"]))
uacur = json.load(open(config["cur_file"]))
prev = config["prev"]
cur = config["cur"]
display(Markdown("# " + config["title"]))
print("Počet platicích učastníků ", prev,": ", len(uaprev))
print("Počet platicích učastníků ", cur,": ", len(uacur))
growth = len(uacur) - len(uaprev)
print("Meziroční růst: ", growth, "(", round(100*(len(uacur)/len(uaprev) - 1)), "%)")
print("Předpověd přištiho roku se stejným růstem:", round(len(uacur)*(len(uacur)/len(uaprev))))

### Table columns

In [ ]:
for key in uacur[1].keys():
    print(key)

In [ ]:
def group_by(uas, key):
    d = {}
    for ua in uas:
        if not ua[key] in d:
            d[ua[key]] = []
        d[ua[key]].append(ua)
    return d

Kolik lidí přihlašili přes stejným účtem?
------------------------------------------

In [ ]:
def by_id(uas):
    return group_by(uas, 'userprofile__user__id')
by_idprev = by_id(uaprev)
by_idcur = by_id(uacur)
id_set_prev = set(by_idprev.keys())
id_set_cur = set(by_idcur.keys())
len(id_set_prev.intersection(id_set_cur))

Kolik lidí přihlašili se stejnou emailovou adresu?
--------------------------------------------------------

In [ ]:
def by_email(uas):
    return group_by(uas, 'userprofile__user__email')
by_emailprev = by_email(uaprev)
by_emailcur = by_email(uacur)
email_set_prev = set(by_emailprev.keys())
email_set_cur = set(by_emailcur.keys())
len(email_set_prev.intersection(email_set_cur))

Kolik lidí přihlašili se jinou (novou) emailovou adresu?
-----------------------------------------------------

In [ ]:
len(email_set_cur - email_set_prev)

Kolik lidí nejezdili pod stejnou emailovou adresu?
--------------------------------------------------

In [ ]:
len(email_set_prev - email_set_cur)

Kolik unikatných telefonních čisel opakovali mezi rokama?
--------------------------------------------------------

In [ ]:
def by_tel(uas):
    return group_by(uas, 'userprofile__telephone')
by_telprev = by_tel(uaprev)
by_telcur = by_tel(uacur)
tel_set_prev = set(by_telprev.keys())
tel_set_cur = set(by_telcur.keys())
len(tel_set_prev.intersection(tel_set_cur))

Kolik lidí přihlašili se jiným (novým) telefonním číslem?
-----------------------------------------------------

In [ ]:
len(tel_set_cur - tel_set_prev)

Kolik lidí nejezdili pod stejným tel číslem?
--------------------------------------------------

In [ ]:
len(tel_set_prev - tel_set_cur)

### Opakované telefonní čisla v ramci jednoho ročniku
Vč. prazdné a vymyšlené čisla

In [ ]:
len(uacur) - len(by_telcur)

In [ ]:
def by_company(uas):
    return group_by(uas, 'team__subsidiary__company__name')
by_firmprev = by_company(uaprev)
by_firmcur = by_company(uacur)
firm_set_prev = set(by_firmprev.keys())
firm_set_cur = set(by_firmcur.keys())

### Analyza schodních jmen

In [ ]:
repeated_names = 0
different_id = 0
for firm in firm_set_cur.intersection(firm_set_prev):
    for ua in by_firmcur[firm]:
        for uap in by_firmprev[firm]:
            if uap['userprofile__user__first_name'] == ua['userprofile__user__first_name'] and ua['userprofile__user__last_name'] == uap['userprofile__user__last_name']:
                repeated_names += 1
                if uap['userprofile__user__id'] != ua['userprofile__user__id']:
                    different_id += 1
                break
print("Opakované jména i firmu:", repeated_names)
print("Procenta minulého ročníku:", round(100*(repeated_names/len(uaprev))), "%")
print("Opakované jména i firmu ale ne účet:", different_id)
different_firm = 0
different_first_name = 0
different_last_name = 0
for id_ in id_set_prev.intersection(id_set_cur):
    if by_idcur[id_][0]['team__subsidiary__company__name'] != by_idprev[id_][0]['team__subsidiary__company__name']:
        different_firm += 1
    if by_idcur[id_][0]['userprofile__user__first_name'] != by_idprev[id_][0]['userprofile__user__first_name']:
        different_first_name += 1
    if by_idcur[id_][0]['userprofile__user__last_name'] != by_idprev[id_][0]['userprofile__user__last_name']:
        different_last_name += 1
print("Stejný účet, jinou firmu:", different_firm)
print("Stejný účet, nebo firmu+jméno:", different_firm+repeated_names, "(", round(100*(different_firm+repeated_names)/len(uaprev)), ")%")
print("Stejný účet, jiné přijmení:", different_last_name)
print("Stejný účet, jiné křestní jméno:", different_first_name)

## Firmy

In [ ]:
print("Počet firem ", prev, ": ", len(firm_set_prev))
print("Počet firem ", cur, ": ", len(firm_set_cur))
print("Průměrný počet učastniku na firmu ", prev, ": ", round(len(uaprev)/len(firm_set_prev), 2))
print("Průměrný počet učastniku na firmu ", cur, ": ", round(len(uacur)/len(firm_set_cur), 2))

### Největší firmy

In [ ]:
%matplotlib inline

In [ ]:
def mk_groups(groups):
    rider_counts = []
    for name, riders in groups.items():
        rider_counts.append((name, len(riders)))
    group_sizes = sorted(rider_counts, key=lambda t: t[1]) 
    import itertools
    group_names = []
    group_size = []
    for n, c in itertools.islice(reversed(group_sizes), 40):
        group_names.append(n)
        group_size.append(c)
    return group_names, group_size
def plot_groups(groups, title):
    group_names, group_size = mk_groups(groups)
    s = pd.Series(group_size, index=group_names)
    s.plot.bar(figsize=(15,8))
    plt.title(title);
plot_groups(by_firmcur, cur)

In [ ]:
plot_groups(by_firmprev, prev)

### Kolik firem s námi zůstali

In [ ]:
len(firm_set_prev.intersection(firm_set_cur))

### Kolik firem odešli?

In [ ]:
len(firm_set_prev - firm_set_cur)

### Kolik nových firem přibili?

In [ ]:
len(firm_set_cur - firm_set_prev)

### Přesun mezi firmama: Stejný telefonní čislo a přijmení, jiná firma

In [ ]:
from collections import Counter
different_firm = set()
old_firms = {}
for tel, uasc in by_telcur.items():
    if tel in by_telprev:
        for uap in by_telprev[tel]:
            for uac in uasc:
                if uac['userprofile__user__last_name'] == uap['userprofile__user__last_name'] and uac['team__subsidiary__company__name'] != uap['team__subsidiary__company__name']:
                    if not uap['team__subsidiary__company__name'] in old_firms:
                        old_firms[uap['team__subsidiary__company__name']] = Counter()
                    old_firms[uap['team__subsidiary__company__name']][uac['team__subsidiary__company__name']] += 1
                    different_firm.add(uac['userprofile__user__id'])
def count_counts(i):
    _, counts = i
    return sum(counts.values())

for old_firm, new_firms in reversed(sorted(old_firms.items(), key=count_counts)):
    print(old_firm, ": (Staré veilikost ", len(by_firmprev[old_firm]), ")")
    for new_firm, count in new_firms.items():
        new = ""
        if new_firm not in by_firmprev:
            new = "| Nová firma ve velikosti " + str(len(by_firmcur[new_firm])) + "!"
        print("  →  Přesovali ",count, "lidi do", new_firm, new)
    if old_firm in by_firmcur:
        print("     Zůstali:", len(by_firmcur[old_firm]))
    else:
        print("     Firma už nejezdí :(")
    print()
print("Celkem:", len(different_firm))

### Největší firmy které odešli
Pozn: Je možné, že některé firmy se přejmenovali?

In [ ]:
lost_firms = {}
for firm in firm_set_prev - firm_set_cur:
    lost_firms[firm] = by_firmprev[firm]
plot_groups(lost_firms, "Odešli")

In [ ]:
def tablify_counts(groups):
    table = []
    for group, members in reversed(sorted(groups.items(), key=lambda i: len(i[1]))):
        table.append([group, len(members)])
    return table

lost_firms_l = tablify_counts(lost_firms)
pd.DataFrame(lost_firms_l, columns=["Název firmy", "Původní velikost"])

### Největší nové firmy
Pozn: Je možné, že některé firmy se přejmenovali?

In [ ]:
new_firms = {}
for firm in firm_set_cur - firm_set_prev:
    new_firms[firm] = by_firmcur[firm]
plot_groups(new_firms, "Nové velké firmy")

In [ ]:
new_firms_l = tablify_counts(new_firms)
pd.DataFrame(new_firms_l, columns=["Název firmy", "Nový velikost"])

### Ztěch firem které snámi zůstali, jak vyvijeli počet učastniků, které sníma jezdili?

In [ ]:
difference_pairs = []
for firm in firm_set_prev.intersection(firm_set_cur):
    difference_pairs.append((firm,len(by_firmcur[firm]) - len(by_firmprev[firm])))
differences = []
firms_by_growth = []
for f,d in sorted(difference_pairs, key=lambda a:a[1]):
    firms_by_growth.append(f)
    differences.append(d)
print("Počet firem se stejným počtem účasniků:", len([x for x in differences if x == 0]))
print("Počet firem se větším počtem účasniků:", len([x for x in differences if x > 0]))
print("Počet firem se menším počtem účasniků:", len([x for x in differences if x < 0]))
s = pd.Series(list(itertools.islice(differences, 40)), index=list(itertools.islice(firms_by_growth, 40)))
s.plot.bar(figsize=(15,8))
plt.title("Změny ve velikost firem (nejhorší)");

In [ ]:
s = pd.Series(list(itertools.islice(reversed(differences), 40)), index=list(itertools.islice(reversed(firms_by_growth), 40)))
s.plot.bar(figsize=(15,8))
plt.title("Změny ve velikost firem (nejlepší)");

## Města

In [ ]:
def by_city(uas):
    return group_by(uas, 'team__subsidiary__city__name')
by_cityprev = by_city(uaprev)
by_citycur = by_city(uacur)
city_set_prev = set(by_cityprev.keys())
city_set_cur = set(by_citycur.keys())
print("Počet měst", prev, ":", len(city_set_prev))
print("Počet měst", cur, ":", len(city_set_cur))
print("Průměrný počet učastniku na město", prev, ":", round(len(uaprev)/len(city_set_prev), 2))
print("Průměrný počet učastniku na město", cur, ":", round(len(uacur)/len(city_set_cur), 2))

### Největší města

In [ ]:
plot_groups(by_citycur, cur)

In [ ]:
plot_groups(by_cityprev, prev)

### Ztěch měst které měli učastnici oba roky, jak vyvijeli počet učastniků, které v ních jezdili?

In [ ]:
difference_pairs = []
for city in city_set_prev.intersection(city_set_cur):
    difference_pairs.append((city, len(by_citycur[city]) - len(by_cityprev[city])))
difference_pairs = sorted(difference_pairs, key=lambda a: a[1])
cities = []
differences = []
for c, d in difference_pairs:
    cities.append(c)
    differences.append(d)
print("Počet měst se stejným počtem účasniků:", len([x for x in differences if x == 0]))
print("Počet měst se větším počtem účasniků:", len([x for x in differences if x > 0]))
print("Počet měst se menším počtem účasniků:", len([x for x in differences if x < 0]))

#### Které města měli menší počet učastníku?

In [ ]:
print("Ubyilo počet učastniků bez města")
for city in city_set_prev.intersection(city_set_cur):
    difference = len(by_citycur[city]) - len(by_cityprev[city])
    if difference < 0:
        print(city)
        print(" ", prev, ":", len(by_cityprev[city]))
        print(" ", cur, ":", len(by_citycur[city]))

### Růst města

In [ ]:
s = pd.Series(differences, index=cities)
s.plot.bar(figsize=(15,7))
plt.title("Růst");

In [ ]:
percent_differences = []
for city, d in difference_pairs:
     percent_differences.append(round(100*(d/len(by_cityprev[city]))))
s = pd.Series(percent_differences, index=cities)
s.plot.bar(figsize=(15,8))
plt.title("Procenta růst");

## Spůsoby platby

In [ ]:
PAY_TYPES = (
        ('mp', 'mPenize - mBank'),
        ('kb', 'MojePlatba'),
        ('rf', 'ePlatby pro eKonto'),
        ('pg', 'GE Money Bank'),
        ('pv', 'Sberbank (Volksbank)'),
        ('pf', 'Fio banka'),
        ('cs', 'PLATBA 24 – Česká spořitelna'),
        ('era', 'Era - Poštovní spořitelna'),
        ('cb', 'ČSOB'),
        ('c', 'Kreditní karta přes GPE'),
        ('bt', 'bankovní převod'),
        ('pt', 'převod přes poštu'),
        ('sc', 'superCASH'),  # Deprecated
        ('psc', 'PaySec'),
        ('mo', 'Mobito'),
        ('uc', 'UniCredit'),
        ('t', 'testovací platba'),

        ('fa', 'faktura mimo PayU'),
        ('fc', 'organizace platí fakturou'),
        ('am', 'člen Klubu přátel Auto*Matu'),
        ('amw', 'kandidát na členství v Klubu přátel Auto*Matu'),
        ('fe', 'neplatí účastnický poplatek '),
        ('', 'nezadano')
    )
PAY_TYPES_DICT = dict(PAY_TYPES)
def by_payment_type(uas):
    grps = group_by(uas, 'payment_type')
    ngrps = {}
    for k,v in grps.items():
        ngrps[PAY_TYPES_DICT[k]] = v
    return ngrps

by_payprev = by_payment_type(uaprev)
by_paycur = by_payment_type(uacur)
pay_set_prev = set(by_payprev.keys())
pay_set_cur = set(by_paycur.keys())
plot_groups(by_paycur, cur)

In [ ]:
plot_groups(by_payprev, prev)

In [ ]:
len(competed_again)

### Jezdili znovu podle spůsob platby

In [ ]:
def plot_repeatition_by_payment_method():
    year_payment_method = []
    repeated = []
    def add_year(uas, year, repeats):
        num_repeats_f = 0
        num_onces_f = 0
        num_repeats_s = 0
        num_onces_s = 0
        ypmf = year + " Fakturou"
        ypms = year + " Samoplatcem"
        for ua in uas:
            if ua['userprofile__user__id'] in repeats:
                repeated.append("Ano")
            else:
                repeated.append("Ne")
            if ua['payment_type'] == 'fc':
                if ua['userprofile__user__id'] in repeats:
                    num_repeats_f += 1
                else:
                    num_onces_f += 1
                year_payment_method.append(ypmf)
            else:
                if ua['userprofile__user__id'] in repeats:
                    num_repeats_s += 1
                else:
                    num_onces_s += 1
                year_payment_method.append(ypms)
        f_tot = num_repeats_f + num_onces_f
        s_tot = num_repeats_s + num_onces_s
        print(f_tot,"lidi jezdili v roce", year, "a platili fakturou")
        print(num_repeats_f,"(", round(100*num_repeats_f/f_tot,1), "% ) Jezdili znovu")
        print(num_onces_f, "(", round(100*num_onces_f/f_tot, 1),"% ) Nejezdili znovu")
        print()
        print(s_tot, "lidi jezdili v roce", year, "a neplatili fakturou")
        print(num_repeats_s,"(", round(100*num_repeats_s/s_tot, 1),"% ) Jezdili znovu")
        print(num_onces_s,"(", round(100*num_onces_s/s_tot, 1),"% ) Nejezdili znovu")
        print()
        print()

    add_year(uaprev, prev, id_set_prev.intersection(id_set_cur))
    add_year(uacur, cur, id_set_prev.intersection(id_set_cur))

            
    data = pd.DataFrame({'Rok/typ platby': year_payment_method, 
                         'Jezdili obě roky?': repeated})

    return pd.crosstab(data['Rok/typ platby'], data['Jezdili obě roky?']).plot.bar(stacked=True, title="Jezdili znovu?", figsize=[15,10])
plot_repeatition_by_payment_method()

#### Podíl ve jednotlivích firmách

In [ ]:
def plot_payment_methods(uas, biggest, title):
    firm_employees = []
    payment_methods = []
    for ua in uas:
        if ua['team__subsidiary__company__name'] in biggest:
            firm_employees.append(str(biggest.index(ua['team__subsidiary__company__name']) + 1).zfill(2) + ". " + ua['team__subsidiary__company__name'])
            if ua['payment_type'] == 'fc':
                payment_methods.append("Fakturou")
            else:
                payment_methods.append("Jinak")
    data = pd.DataFrame({'Firmy': firm_employees, 
                         'Spůsob plátby': payment_methods})

    return pd.crosstab(data.Firmy, data['Spůsob plátby']).plot.bar(stacked=True, title=title, figsize=[15,10])
plot_payment_methods(uacur, list(reversed(firms_by_growth[-40:])), "Podíl způsob faktury " + cur + "- firmy se největším růstem")

In [ ]:
plot_payment_methods(uaprev, list(reversed(firms_by_growth[-40:])), "Podíl způsob faktury " + prev + "- firmy se největším růstem")

In [ ]:
plot_payment_methods(uaprev, firms_by_growth[:40], "Podíl způsob faktury " + prev + "- firmy se největším ztratem")

In [ ]:
plot_payment_methods(uacur, firms_by_growth[:40], "Podíl způsob faktury " + cur + "- firmy se největším ztratem")

In [ ]:
plot_payment_methods(uacur, mk_groups(by_firmcur)[0][:40], "Podíl způsob faktury " + cur + "- největší firmy vůbec")

In [ ]:
plot_payment_methods(uaprev, mk_groups(by_firmprev)[0][:40], "Podíl způsob faktury " + prev + "- největší firmy vůbec")

### Firmy se samoplatci

In [ ]:
def get_self_payers(payment_type_groups):
    self_payers = []
    non_self_pay_types = ['neplatí účastnický poplatek ', 'organizace platí fakturou']
    for k,v in payment_type_groups.items():
        if k not in non_self_pay_types:
            self_payers.extend(v)
    return self_payers
plot_groups(by_company(get_self_payers(by_paycur)), cur)

In [ ]:
plot_groups(by_company(get_self_payers(by_payprev)), prev)

#### Firma s největší počet samoplatci

In [ ]:
def plot_company_payment_types(firm):
    plot_groups(by_payment_type(by_firmcur[firm]), firm + " " + cur + " - podle typ platby")
def plot_company_payment_types_prev(firm):
    plot_groups(by_payment_type(by_firmprev[firm]), firm + " " + prev + " - podle typ platby")
def get_biggest(uas):
    n = 0
    biggest = ''
    for k,v in uas.items():
        if len(v) > n:
            n = len(v)
            biggest = k
    return biggest
biggest_mixed = get_biggest(by_company(get_self_payers(by_payprev)))
plot_company_payment_types(biggest_mixed)

In [ ]:
plot_company_payment_types_prev(biggest_mixed)

### Firmy, které platili jen některé účastnici

In [ ]:
mixed_firms = {}
for firm, uas in by_company(get_self_payers(by_paycur)).items():
    if len(by_firmcur[firm]) != len(uas):
        mixed_firms[firm] = by_firmcur[firm]
plot_groups(mixed_firms, cur)

#### Největší firma

In [ ]:
plot_company_payment_types_prev(get_biggest(by_firmcur))

In [ ]:
plot_company_payment_types(get_biggest(by_firmcur))

#### Největší nová firma

In [ ]:
plot_company_payment_types(get_biggest(new_firms))

### Jak se vyvijelo typ platby u firem které stratili učastníků?

In [ ]:
plot_company_payment_types_prev(firms_by_growth[0])

In [ ]:
plot_company_payment_types(firms_by_growth[0])

In [ ]:
plot_company_payment_types_prev(firms_by_growth[1])

In [ ]:
plot_company_payment_types(firms_by_growth[1])

In [ ]:
plot_company_payment_types_prev(firms_by_growth[2])

In [ ]:
plot_company_payment_types(firms_by_growth[2])

In [ ]:
plot_company_payment_types_prev(firms_by_growth[3])

In [ ]:
plot_company_payment_types(firms_by_growth[3])

In [ ]:
plot_company_payment_types_prev(firms_by_growth[4])

In [ ]:
plot_company_payment_types(firms_by_growth[4])

### Jak vyvijelo typy platby u firem, u které přibili účastnici?

In [ ]:
plot_company_payment_types_prev(firms_by_growth[-1])

In [ ]:
plot_company_payment_types(firms_by_growth[-1])

In [ ]:
plot_company_payment_types_prev(firms_by_growth[-2])

In [ ]:
plot_company_payment_types(firms_by_growth[-2])

In [ ]:
plot_company_payment_types_prev(firms_by_growth[-3])

In [ ]:
plot_company_payment_types(firms_by_growth[-3])

In [ ]:
plot_company_payment_types_prev(firms_by_growth[-4])

In [ ]:
plot_company_payment_types(firms_by_growth[-4])

In [ ]:
plot_company_payment_types_prev(firms_by_growth[-5])

In [ ]:
plot_company_payment_types(firms_by_growth[-5])

## Velikost triček

In [ ]:
plot_groups(group_by(uacur, 't_shirt_size__name'), cur)

In [ ]:
plot_groups(group_by(uaprev, 't_shirt_size__name'), prev)

In [ ]:
by_shirtcur = group_by(uacur, 't_shirt_size__name')
sizes = ['XXXL', 'XXL', 'XL', 'L', 'M', 'S', 'XS']
def get_size(ua):
    s = len(sizes)
    for size in sizes:
        if ua['t_shirt_size__name'].endswith(' ' + size):
            return s
        s += 1
    return s
larger = 0
smaller = 0
same = 0
total = 0

for k,v in by_idcur.items():
    if k in by_idprev:
        ps = get_size(by_idprev[k][0])
        cs = get_size(v[0])
        total += 1
        if cs > ps:
            larger += 1
        if cs == ps:
            same += 1
        if cs < ps:
            smaller += 1
print("Větší", larger, "(", round(100*larger/total),"% ) Stejné", same, "(", round(100*same/total),"% ) Menší", smaller, "(", round(100*smaller/total),"% )")